In [1]:
# Set up modules for Google functionality
from google.cloud import bigquery # To run BQ statements
from google_auth_oauthlib import flow # To authorise as user
from googleapiclient.discovery import build # To pull in from sheets, slides etc. API
from google.auth.transport.requests import Request

# Display
import pprint

# Operating system stuff
import pickle
import os.path
import sys

# Data handling
import json
import requests
from pandas import read_csv
from pandas import datetime

# Stats, models, datasheets
import pandas as pd
import pyreadstat

# Visualisation
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib_venn # For venn diagrams
from pandas.plotting import autocorrelation_plot

# Network graphs
import networkx as nx


# Misc
from xlsxwriter.utility import xl_rowcol_to_cell # Used to create cell references
import itertools

# Load custom scripts in reusable_code folder
sys.path.append(r'/home/jupyter/reusable_code')

import google_api_functions as gaf


In [2]:
creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/')
bq = bigquery.Client(project='itv-bde-analytics-dev',credentials=creds)

In [4]:
values,vipdf=gaf.read_google_sheets_as_rows('1JTSA-oOdVjGeoTGECBJ-rZRdReYtfRm41HvpLUuMHzk','Sheet1',creds,header_row=0)

In [5]:
dataset=bq.dataset('britbox_sandbox')
table_ref = dataset.table("VIP_Vouchers_2019")


newcol_names={x:x.replace(" ", "_").replace("/","").replace("?","").replace("-","").replace(".","") for x in metro_df.columns}
df_for_bq=vipdf.rename(columns=newcol_names)

try:
    bq.delete_table(table_ref)
except:
    pass
job = bq.load_table_from_dataframe(df_for_bq, table_ref)

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

In [20]:
query='''select a.Description as BBID,
max(case when b.user_id is not null then 1 else 0 end) as Used_in_last_mo
from
(select distinct Description
from `itv-bde-analytics-dev.britbox_sandbox.VIP_Vouchers_2019`) a
left join
(select distinct coalesce(TRIM(JSON_EXTRACT(raw,'$.user.itv_id'),"\""),TRIM(JSON_EXTRACT(raw,'$.user.id'),"\"")) as user_id
from `itv-bde-svod-prd.svod.svod_page_raw_in` 
where _PARTITIONTIME>='2020-04-01') b
on a.Description=b.user_id
group by 1
'''

query='''select * from `itv-bde-analytics-dev.britbox_sandbox.Vip_subs_usage`'''
used_df = bq.query(query).to_dataframe()
used_df.head()

In [21]:
used_df['Used_in_last_mo'].sum()

In [22]:
gaf.Write_whole_df_to_gsheet(creds, used_df, '1JTSA-oOdVjGeoTGECBJ-rZRdReYtfRm41HvpLUuMHzk', 'Used in Apr-May10', valueInputOption='RAW', append_overwrite='overwrite', headers='Y', topleftcell='A1')
    
 